# 📊 Análisis Comparativo de Resultados
Este notebook consolida los resultados guardados en archivos `.json` generados por los distintos notebooks del proyecto y produce al menos **dos gráficos por análisis** comparativo.

Se incluyen análisis como:
- ToT vs RAG (evaluación)
- Study-Plan Generator vs AI Tutor
- Robustez del generador (fallos por iteración)
- Iteraciones en MCTS vs Calidad de respuesta


In [ ]:
import json
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

RESULTS_DIR = Path("../test")
files = list(RESULTS_DIR.glob("*.json"))
print("Archivos de resultados detectados:")
for f in files:
    print("-", f.name)

In [ ]:
# Cargar todos los archivos de resultados
data = {}
for f in files:
    try:
        with open(f, "r", encoding="utf-8") as file:
            contenido = json.load(file)
            data[f.name] = contenido
    except Exception as e:
        print(f"Error cargando {f.name}: {e}")

## 📘 Comparación ToT vs RAG (evaluación por JuezIA)

In [ ]:
# Buscar archivo que contenga comparacion_tot_vs_rag
import pandas as pd

for nombre, contenido in data.items():
    if "tot_vs_rag" in nombre:
        df = pd.DataFrame([
            {"método": "ToT", "evaluacion": contenido['tot']['evaluacion']},
            {"método": "RAG", "evaluacion": contenido['rag']['evaluacion']}
        ])
        print(df)
        plt.figure(figsize=(8, 4))
        sns.barplot(x="método", y=df.index, data=df)
        plt.title("Comparación ToT vs RAG (JuezIA - longitud del texto evaluativo)")
        plt.ylabel("Longitud de evaluación")
        plt.xlabel("Método")
        plt.bar(df['método'], df['evaluacion'].apply(lambda x: len(x)))
        plt.show()

## 📚 Comparación Study-Plan Generator vs AI Tutor

In [ ]:
# Buscar archivo de comparación de planes de estudio
planes = []
for nombre, contenido in data.items():
    if "comparacion_planes" in nombre:
        for item in contenido:
            planes.append({
                "método": item["metodo"],
                "evaluacion": item["evaluacion"]
            })

df_planes = pd.DataFrame(planes)
plt.figure(figsize=(8, 4))
sns.countplot(data=df_planes, x="método")
plt.title("Frecuencia de planes generados por método")
plt.show()

df_planes["longitud"] = df_planes["evaluacion"].apply(lambda x: len(x))
plt.figure(figsize=(8, 4))
sns.boxplot(data=df_planes, x="método", y="longitud")
plt.title("Distribución de longitud de evaluación por método")
plt.ylabel("Longitud de evaluación")
plt.show()

## 🧪 Robustez del Study-Plan Generator (fallos vs intentos)

In [ ]:
# Cargar simulación de fallos
for nombre, contenido in data.items():
    if "fallos_study_plan" in nombre:
        intentos = contenido["historial_intentos"]
        plt.figure(figsize=(8, 4))
        sns.histplot(intentos, bins=range(1, max(intentos)+2), kde=False)
        plt.title("Histograma de intentos por generación válida")
        plt.xlabel("Intentos hasta plan válido")
        plt.ylabel("Frecuencia")
        plt.show()

        plt.figure(figsize=(6, 4))
        sns.boxplot(x=intentos)
        plt.title("Distribución de intentos")
        plt.xlabel("Intentos")
        plt.show()

## 🧠 Iteraciones en MCTS vs Evaluación de Calidad

In [ ]:
# Visualizar calidad en función de iteraciones
mcts_resultados = []
for nombre, contenido in data.items():
    if "mcts_iteraciones" in nombre:
        for item in contenido:
            mcts_resultados.append({
                "iteraciones": item["iteraciones"],
                "evaluacion_len": len(item["evaluacion"])
            })

df_mcts = pd.DataFrame(mcts_resultados)

plt.figure(figsize=(8, 4))
sns.barplot(data=df_mcts, x="iteraciones", y="evaluacion_len")
plt.title("Evaluación (longitud) vs iteraciones MCTS")
plt.ylabel("Longitud de la evaluación del juez")
plt.xlabel("Iteraciones de MCTS")
plt.show()

plt.figure(figsize=(6, 4))
sns.lineplot(data=df_mcts, x="iteraciones", y="evaluacion_len", marker="o")
plt.title("Curva de mejora aparente por iteraciones")
plt.show()